## Sim Demo

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from env.imports import *

/ext3/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-09-16 13:01:48.546844: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-16 13:01:50.426994: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-16 13:01:52.464265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  import numpy.matlib as npm

  au

In [3]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run)
importlib.reload(models.train_val)
importlib.reload(data.data_utils)


<module 'data.data_utils' from '/scratch/kl5565/GeneEx2Conn-master/data/data_utils.py'>

#### Check job specs

In [8]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 2.9%
RAM Usage: 8.1%
Available RAM: 346.5G
Total RAM: 377.1G
52.4G


In [9]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")


0
Available GPUs: 1
GPU 0: Tesla V100-PCIE-32GB - Memory Allocated: 0.00 GB


In [10]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

GPUtil.getAvailable()

# if a number is seen a GPU is available
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

    GPUtil.showUtilization()

XGBoost version: 2.0.3
cupy version: 13.1.0
GPU found 0
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [11]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Simulation tests <a id="sims"></a>

### Model Parameter Counts

#### **Linear Models**
- **PCA Bilinear:** 730  _(27 PCs)_
- **PLS Bilinear:** 101  _(10 PLS components)_
- **Bilinear Low-rank:** 73,800  _(rank 10)_
- **PLS MLP:** 158,993  _(10 PLS components, including PLS projection matrices)_
- **PCA MLP:** 47,873  _(27 PCs, 2-layer)_

---

#### **MLP and SMT Models**

#### 2-Layer Models
- **MLP:** 3,812,609
- **SMT:** 1,399,947
- **MLP w/ CLS:** 3,814,145
- **SMT w/ CLS:** 1,405,579

#### 3-Layer Models
- **MLP:** 7,723,777
- **SMT:** 2,162,315
- **MLP w/ CLS:** 7,726,849
- **SMT w/ CLS:** 2,173,067

---

### Coord MLP Parameter Counts
- **[32]:** 321
- **[64, 32]:** 2,753
- **[128, 64]:** 9,601
- **[256, 128]:** 35,685
- **[512, 256, 128]:** 169,729

In [13]:
torch._dynamo.reset()

In [ ]:

#cnn
single_sim_run(
    dataset='HCP',
    cv_type='spatial',
    random_seed=42,
    model_type='cnn1d',                 # ← 使用你的 CNN
    use_gpu=True,
    feature_type=[{'transcriptome': None}],  # 先用最简单特征
    connectome_target='FC',
    omit_subcortical=True, parcellation='S456', hemisphere='both',
    gene_list='0.2',
    search_method=('wandb','mse',10),
    track_wandb=True,
    skip_cv=False
)

Number of components for 95% variance PCA: 27
Number of 1s: 36662, Number of 0s: 171274, Class balance (1s): 0.176
Number of 1s: 11988, Number of 0s: 195948, Class balance (1s): 0.058
X shape: (339, 7380)
X_pca shape: (339, 27)
X_pca_full shape: (339, 455)
X_cell_types_Jorstad shape: (339, 24)
X_cell_types_LakeDFC shape: (339, 18)
X_cell_types_LakeVIS shape: (339, 18)
Y_sc shape: (339, 339)
Y_sc_spectralL shape: (339, 399)
Y_sc_spectralA shape: (339, 400)
Y_fc shape: (339, 339)
Coordinates shape: (339, 3)
Y shape (339, 339)
Network coverage: 100.0% of regions
Network sizes: [84, 84, 84, 87]
feature_name: transcriptome, processing_type: None
features ['transcriptome']
Feature matrix, X, generated... expanding to pairwise dataset


  return LooseVersion(v) >= LooseVersion(check)

ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kl5565 (kl5565-new-york-university). Use `wandb login --relogin` to force relogin


Create sweep with ID: 36uuvq52
Sweep URL: https://wandb.ai/kl5565-new-york-university/gx2conn/sweeps/36uuvq52
Initialized sweep with ID: 36uuvq52
2
3
4


wandb: Agent Starting Run: 4tmxc4h1 with config:
wandb: 	batch_size: 256
wandb: 	binarize: False
wandb: 	channels: [16, 32, 64]
wandb: 	dropout_rate: 0.1
wandb: 	epochs: 3
wandb: 	fc_hidden: 128
wandb: 	input_dim: 14760
wandb: 	kernel_sizes: [7, 5, 3]
wandb: 	learning_rate: 0.0005
wandb: 	weight_decay: 0.001
ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
  self._sweep_config = config_util.dict_from_config_file(

  self.comm = Comm(**args)

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
  from IPython.core.display import HTML, display  # type: ignore



Processing inner fold 0


...

Processing inner fold 0
Using device: cuda
GPU 0: Tesla V100-PCIE-32GB - Memory Allocated: 8.58 GB
Best val loss so far at epoch 1: 0.0463
Best val loss so far at epoch 2: 0.0457
Best val loss so far at epoch 3: 0.0455


  from IPython.core.display import display



avg_train_loss,█▁▁
best_val_loss,█▃▁
epoch,▁▅█
learning_rate,▁▁▁
train_batch_loss,█▇▁▅▆▃
val_loss,█▃▁
avg_train_loss,0.02627
best_val_loss,0.04554
epoch,3
learning_rate,0.0001
train_batch_loss,0.02455


Run n0xdrqai errored:
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/scratch/kl5565/GeneEx2Conn-master/sim/sim.py", line 254, in train_sweep_wrapper
    return train_sweep_torch(
           ^^^^^^^^^^^^^^^^^^
  File "/scratch/kl5565/GeneEx2Conn-master/sim/sim_utils.py", line 368, in train_sweep_torch
    history = model.fit(dataset, train_indices_expanded, test_indices_expanded)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/scratch/kl5565/GeneEx2Conn-master/models/cnn_models.py", line 172, in fit
    return train_model(
           ^^^^^^^^^^^^
  File "/scratch/kl5565/GeneEx2Conn-master/models/train_val.py", line 55, in train_model
    predictions, targets = model.predict(val_loader)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ext3/miniconda3/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 116,

Processing inner fold 0
Using device: cuda
GPU 0: Tesla V100-PCIE-32GB - Memory Allocated: 13.99 GB


In [ ]:
!nvidia-smi